### Imports and connection setup

In [1]:
import hana_ml
import hana_ml.dataframe as dataframe

from hana_ml.algorithms.pal.unified_classification import UnifiedClassification
from hana_ml.algorithms.pal.partition import train_test_val_split

from hana_ml.artifacts.generators import AMDPGenerator

print(hana_ml.__version__)

2.11.22010700


In [ ]:
hana_url = 'xxx'
port = 'xx'
user = 'xx'
password = 'xxx'

print("URL: " + hana_url)
print("User: " + user)

In [4]:
conn = dataframe.ConnectionContext("{}".format(hana_url),int("{}".format(port)), "{}".format(user), "{}".format(password), encrypt="true", sslValidateCertificate="false")
conn.hana_version()

'2.00.059.01.1638875844 (fa/hana2sp05)'

### Connect to the data

We will generate an SAP HANA DataFrame from our source data. Please note the usage of the placeholder clause in the SQL statement, since our source is an SAP BW query with ...

In [5]:
df_hana_source = dataframe.DataFrame(conn,'SELECT "4ZANON_03_ID" as ID, "4ZANON_03_GENDER" as GENDER, "4ZANON_03_LASTNAME" as LASTNAME, "4ZANON_03_EDUCATION" as EDUCATION, "4ZANON_03_REGION" as REGION, "4ZANON_03_START_YEAR" as START_YEAR, "4ZANON_03_ZIPCODE" as ZIPCODE, "T_LEVEL", ("5I3Y60GN5JCU6PGWX331P8G0G") AS "SALARY" FROM "_SYS_BIC"."system-local.bw.bw2hana.query.zanon_03/Z_QRY_EXIT_001"(\'PLACEHOLDER\' = (\'$$ZV_REGION_IP$$\', \'EMEA\'))') 
df_hana_source = df_hana_source.cast('ID', 'INTEGER')

In [6]:
df_hana_source.collect()

,ID,GENDER,LASTNAME,EDUCATION,REGION,START_YEAR,ZIPCODE,T_LEVEL,SALARY
0,197063,f,Drock,College,EMEA,2014,5005,T1,44391.0
1,130361,f,Barer,9th-12th,EMEA,2000,5106,T3,99042.0
2,138163,f,Rizzio,College,EMEA,2006,5004,T3,88915.0
3,168770,f,Nawda,Bachelor,EMEA,2014,5204,T2,53938.0
4,122025,f,Muellner,Prof-school,EMEA,1997,5005,T4,122612.0
...,...,...,...,...,...,...,...,...,...
66519,114696,f,,HS-grad,EMEA,1996-2000,51xx,T3-T5,100985.0
66520,167619,f,,Bachelor,EMEA,2011-2015,51xx,T1-T2,46803.0
66521,174626,f,,HS-grad,EMEA,2006-2010,50xx,T1-T2,50075.0
66522,164289,m,,College,EMEA,2011-2015,51xx,T3-T5,88725.0


Generate seperate data set for Training, Testing and Validation of our model. We will skipp validation here, since the ML part is not the focus of this demo.

In [7]:
train, test, valid = train_test_val_split(data=df_hana_source, training_percentage = 0.8, validation_percentage = 0, testing_percentage = 0.2, random_seed = 41)

### ML Training

We will leverage the AMDP generator to get our AMDP/ABAP automatically generated. This requires the SQL logging to be turned on. 

In [8]:
conn.sql_tracer.enable_sql_trace(True)
conn.sql_tracer.enable_trace_history(True)

In [9]:
uni_hgbt = UnifiedClassification(func='HybridGradientBoostingTree')

In [10]:
uni_hgbt.fit(data = train,
             key= 'ID',
             label= 'T_LEVEL',
             features=['GENDER', 'EDUCATION', 'REGION', 'START_YEAR', 'ZIPCODE', 'SALARY']
             )

INFO:hana_ml.ml_base:Executing SQL: CREATE LOCAL TEMPORARY COLUMN TABLE "#PAL_UNIFIED_CLASSIFICATION_MATERIALIZED_INPUT_F14494DB_7697_4392_8793_464A1C921B8D" AS (SELECT "ID", "GENDER", "EDUCATION", "REGION", "START_YEAR", "ZIPCODE", "SALARY", "T_LEVEL" FROM (SELECT a.* FROM #PAL_PARTITION_DATA_TBL_026B3402_7238_11EC_8EA8_0242AC110002 a inner join #PAL_PARTITION_RESULT_TBL_026B3402_7238_11EC_8EA8_0242AC110002 b        on a."ID" = b."ID" where b."PARTITION_TYPE" = 1) AS "DT_4")
INFO:hana_ml.ml_base:Executing SQL: DO (IN in_0 TABLE ("ID" INT, "GENDER" NVARCHAR(100), "EDUCATION" NVARCHAR(50), "REGION" NVARCHAR(100), "START_YEAR" NVARCHAR(100), "ZIPCODE" NVARCHAR(10), "SALARY" DOUBLE, "T_LEVEL" NVARCHAR(5)) => "#PAL_UNIFIED_CLASSIFICATION_MATERIALIZED_INPUT_F14494DB_7697_4392_8793_464A1C921B8D")
BEGIN
DECLARE param_name VARCHAR(5000) ARRAY;
DECLARE int_value INTEGER ARRAY;
DECLARE double_value DOUBLE ARRAY;
DECLARE string_value VARCHAR(5000) ARRAY;
param_name[1] := N'FUNCTION';
int_value[1]

In [11]:
result = uni_hgbt.predict(
            data=test.select('ID', 'GENDER', 'EDUCATION', 'REGION', 'START_YEAR', 'ZIPCODE', 'SALARY'),
            features=['GENDER', 'EDUCATION', 'REGION', 'START_YEAR', 'ZIPCODE', 'SALARY' ], 
            key= 'ID')

INFO:hana_ml.ml_base:Executing SQL: CREATE LOCAL TEMPORARY COLUMN TABLE "#PAL_UNIFIED_CLASSIFICATION_PREDICT_MATERIALIZED_INPUT_BA1B51B7_9C57_47DE_B0B0_5F0D7EB4A0FC" AS (SELECT "ID", "GENDER", "EDUCATION", "REGION", "START_YEAR", "ZIPCODE", "SALARY" FROM (SELECT "ID", "GENDER", "EDUCATION", "REGION", "START_YEAR", "ZIPCODE", "SALARY" FROM (SELECT a.* FROM #PAL_PARTITION_DATA_TBL_026B3402_7238_11EC_8EA8_0242AC110002 a inner join #PAL_PARTITION_RESULT_TBL_026B3402_7238_11EC_8EA8_0242AC110002 b        on a."ID" = b."ID" where b."PARTITION_TYPE" = 2) AS "DT_5") AS "DT_27")
INFO:hana_ml.ml_base:Executing SQL: DO (IN in_0 TABLE ("ID" INT, "GENDER" NVARCHAR(100), "EDUCATION" NVARCHAR(50), "REGION" NVARCHAR(100), "START_YEAR" NVARCHAR(100), "ZIPCODE" NVARCHAR(10), "SALARY" DOUBLE) => "#PAL_UNIFIED_CLASSIFICATION_PREDICT_MATERIALIZED_INPUT_BA1B51B7_9C57_47DE_B0B0_5F0D7EB4A0FC",
    IN in_1 TABLE ("ROW_INDEX" INT, "PART_INDEX" INT, "MODEL_CONTENT" NCLOB) => "#PAL_UNIFIED_CLASSIFICATION_MODEL_0_0

### Generate ABAP code template

In [12]:
generator = AMDPGenerator(project_name="T-Level", version="1", connection_context=conn, outputdir="out/")
generator.generate()

### Preview results and clean up trace configuration

In [13]:
result.collect().head(10)

,ID,SCORE,CONFIDENCE,REASON_CODE
0,175311,T3,0.621335,"[{""attr"":""ZIPCODE"",""pct"":49.865572691817626,""v..."
1,124967,T4,0.819824,"[{""attr"":""SALARY"",""pct"":54.023751902260472,""va..."
2,180614,T2,0.649066,"[{""attr"":""SALARY"",""pct"":55.728970994351471,""va..."
3,193715,T1,0.908541,"[{""attr"":""SALARY"",""pct"":66.982351435842901,""va..."
4,180745,T2,0.640329,"[{""attr"":""SALARY"",""pct"":55.74549077390504,""val..."
5,162019,T3,0.676600,"[{""attr"":""ZIPCODE"",""pct"":49.867339704612959,""v..."
6,105997,T4,0.809167,"[{""attr"":""SALARY"",""pct"":55.612061270747596,""va..."
7,111185,T4,0.798492,"[{""attr"":""SALARY"",""pct"":60.045352477726944,""va..."
8,106242,T3,0.648997,"[{""attr"":""ZIPCODE"",""pct"":49.865572691817626,""v..."
9,188652,T2,0.527376,"[{""attr"":""SALARY"",""pct"":50.508305924541617,""va..."


In [14]:
conn.sql_tracer.clean_trace_history()

In [15]:
conn.sql_tracer.enable_sql_trace(False)
conn.sql_tracer.enable_trace_history(False)